# Homework 2

## FINM 37400 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

In [138]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.4f}'.format)

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (A): US Treasuries in November 2008

## Data
* Use the data file `treasury_ts_2015-08-15.xlsx`.
* Examine the treasure issues with `kytreasno` of `204046` and `204047`. These are the bond and note (respectively) which mature on 2015-08-15.
* Look at the data on 2008-11-04.



In [105]:
treasuries = pd.read_excel("treasury_ts_2015-08-15.xlsx")
treasuries.columns = treasuries.iloc[0]

treasuries.drop(0,inplace=True)
treasuries.set_index('kytreasno',inplace=True)

treasuries

,204046,204047,206524
kytreasno,,,
kycrspid,20150815.110620,20150815.204250,20150815.200250
crspid,20150815.110620,20150815.204250,20150815.200250
tcusip,912810DS,912828EE,912828TK
tdatdt,1985-08-15 00:00:00,2005-08-15 00:00:00,2012-08-15 00:00:00
tmatdt,2015-08-15 00:00:00,2015-08-15 00:00:00,2015-08-15 00:00:00
iwhy,1,1,1
tcouprt,10.6250,4.2500,0.2500
tnippy,2,2,2
tvalfc,5.3125,2.1250,0.1250


In [106]:
treasury_database = pd.read_excel("treasury_ts_2015-08-15.xlsx",sheet_name="database")

treasury_database.set_index("kytreasno",inplace=True)
treasury_specific = treasury_database.loc[[204046,204047]]

display(treasury_specific)

,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
kytreasno,,,,,,,,,,,,,,,,
204046,20150815.1106,1989-05-02,116.0625,116.3125,116.1875,M,R,2.2307,0.0058,0.0002,3662.9635,6470.0000,7150.0000,0.0000,NaN,NaN
204046,20150815.1106,1989-05-03,116.1875,116.4375,116.3125,M,R,2.2600,0.0013,0.0002,3664.5035,6470.0000,7150.0000,0.0000,NaN,NaN
204046,20150815.1106,1989-05-04,115.5938,115.8438,115.7188,M,R,2.2894,-0.0048,0.0002,3651.5041,6470.0000,7150.0000,0.0000,NaN,NaN
204046,20150815.1106,1989-05-05,116.2188,116.4688,116.3438,M,R,2.3187,0.0055,0.0002,3663.1657,6470.0000,7150.0000,0.0000,NaN,NaN
204046,20150815.1106,1989-05-08,115.7188,115.9688,115.8438,M,R,2.4068,-0.0035,0.0002,3650.0999,6470.0000,7150.0000,0.0000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204047,20150815.2043,2012-07-30,111.8203,111.8516,111.8359,M,I,1.9382,0.0009,0.0000,1039.7147,25114.0000,32470.0000,0.0000,NaN,NaN
204047,20150815.2043,2012-07-31,111.9141,111.9531,111.9336,M,I,1.9499,0.0010,0.0000,1038.7617,25114.0000,32470.0000,0.0000,NaN,NaN
204047,20150815.2043,2012-08-01,111.7969,111.8281,111.8125,M,I,1.9615,-0.0010,0.0000,1037.7138,25114.0000,32470.0000,0.0000,NaN,NaN


In [108]:
treasury_specific.reset_index(inplace=True)
treasury_specific.set_index("caldt",inplace=True)
quote_date = pd.to_datetime("2008-11-04")
treasury_quote_date = treasury_specific.loc[quote_date][['kytreasno','tdbid','tdask','tdnomprc','tdaccint','tdretnua','tdyld','tdduratn']]

treasury_quote_date.reset_index(inplace=True)
treasury_quote_date.set_index('kytreasno',inplace=True)
treasury_quote_date['tdyld'] = treasury_quote_date['tdyld']*365.25 
coupon_rates = [treasuries[204046].loc['tcouprt'],treasuries[204047].loc['tcouprt']]
treasury_quote_date['tcouprt'] = coupon_rates

treasury_quote_date

,caldt,tdbid,tdask,tdnomprc,tdaccint,tdretnua,tdyld,tdduratn,tcouprt
kytreasno,,,,,,,,,
204046,2008-11-04,141.8594,141.8906,141.8750,2.3387,0.0097,0.0358,1910.3079,10.6250
204047,2008-11-04,105.9531,105.9844,105.9688,0.9355,0.0116,0.0324,2168.0166,4.2500


In [93]:
from scipy.optimize import fsolve
import math

def get_ytm(price,coupon,ttm,freq=2):
    def obj_func(ytm):    
        disc_sums = 0
        num_cashflows = math.ceil(freq*ttm)
        if(num_cashflows==1):
            eq = price - (100+coupon/freq)/(math.pow((1+ytm/freq),freq*ttm))
        
        else:    
            for i in range(1,num_cashflows):
                disc_sums += (coupon/freq)/(math.pow((1+ytm/freq),(freq*ttm-i)))
            eq = price - disc_sums - (100+coupon/freq)/(math.pow((1+ytm/freq),freq*ttm))
    
        return eq
    return fsolve(obj_func,0.1)[0]/365.25        


## 1.1 The situation

Make a chart comparing the issues in the following features, (as of Nov 4, 2008.)
* coupon rate
* bid
* ask
* accrued interest
* dirty price
* duration (quoted in years, not days, assuming 365.25 days per year.)
* modified duration
* YTM



In [109]:
treasury_comparison = pd.DataFrame()
treasury_comparison
treasury_comparison['kytreasno'] = [204046,204047]
treasury_comparison.set_index('kytreasno',inplace=True)

treasury_comparison[['coupon rate','bid','ask','accrued interest']] = treasury_quote_date[['tcouprt','tdbid','tdask','tdaccint']]
treasury_comparison['dirty price'] = (treasury_quote_date['tdbid']+treasury_quote_date['tdask'])/2 + treasury_quote_date['tdaccint']
treasury_comparison['duration'] = treasury_quote_date['tdduratn']/(365.25)

maturity_date = pd.to_datetime("2015-08-15")
ttm = (maturity_date.timestamp()-quote_date.timestamp())/(60*60*24*365.25)

treasury_comparison['YTM'] = [get_ytm(treasury_comparison['dirty price'].iloc[0],treasury_comparison['coupon rate'].iloc[0],ttm)*365.25,
                              get_ytm(treasury_comparison['dirty price'].iloc[1],treasury_comparison['coupon rate'].iloc[1],ttm)*365.25]
treasury_comparison['modified duration'] = treasury_comparison['duration']/(1+treasury_comparison['YTM']/2)

treasury_comparison

,coupon rate,bid,ask,accrued interest,dirty price,duration,YTM,modified duration
kytreasno,,,,,,,,
204046,10.6250,141.8594,141.8906,2.3387,144.2137,5.2301,0.0361,5.1374
204047,4.2500,105.9531,105.9844,0.9355,106.9042,5.9357,0.0326,5.8404


## 1.2 Hedge Ratio

Suppose you are building a trade to go long $n_i$ bonds (`204046`) and short $n_j$ notes (`204047`).

We can find an equation for $n_j$ in terms of $n_i$ such that the total holdings will have duration equal to zero. (Having zero duration also means have zero dollar duration, if helpful.)

Notation:
* $n_i$: number of bonds purchased (or sold)
* $D_i$: duration of bond $i$
* $D_{\$,i}$: dollar duration of bond $i$, equal to $p_iD_i$

If we want the total duration of our holdings to be zero, then we need to size the trade such that $n_i$ and $n_j$ satisfy,

$$0 = n_iD_{\$,i} + n_jD_{\$,j}$$

$$n_j = -n_i\frac{D_{\$,i}}{D_{\$,j}}$$

Suppose you will use \\$1mm of capital, leveraged 50x to buy \\$50mm of the bonds (`204046`).

Use the ratio above to short a number of notes (`204047`) to keep zero duration.

Report the number of bonds and notes of your position, along with the total dollars in the short position.

In [110]:
n1 = 5*1e7/treasury_comparison['dirty price'].loc[204046]
n2 = -n1*(treasury_comparison['duration'].loc[204046]*treasury_comparison['dirty price'].loc[204046]/(treasury_comparison['duration'].loc[204047]*treasury_comparison['dirty price'].loc[204047]))

positions = pd.DataFrame()
positions['kytreasno'] = [204046,204047]
positions.set_index('kytreasno',inplace=True)
positions['Position in treasury'] = [n1,n2]
positions['Dollars in position (in millions)'] = positions['Position in treasury']*treasury_comparison['dirty price']/1e6

positions

,Position in treasury,Dollars in position (in millions)
kytreasno,,
204046,346707.8068,50.0000
204047,-412112.6579,-44.0566


## 1.3 Profit Opportunity

Using the concept of **modified duration**, how much profit or loss (PnL) would you expect to make for every basis point of convergence in the spread? Specifically, assume the convergence is symmetric: the bond's (`204046`) ytm goes down 0.5bp and the note (`204047`) ytm goes up 0.5bp.

Describe the PnL you would expect to achieve on your position should this happen. Specify the PnL of the long position, the short position, and the net total.

$D_{ytm} = -\frac{1}{P}*\frac{dP}{d(ytm)}$

This implies that:  $ dP = -P * D_{ytm} * d(ytm) $

Therefore, $ PnL = dP * position = position * (-P * D_{ytm} * d(ytm)) $

In [111]:
def PnL(position,price,modified_duration,bp):
    return -position*price*modified_duration*bp/10000

returns = pd.DataFrame()
returns['kytreasno'] = [204046,204047,'Total PnL']
returns.set_index('kytreasno',inplace=True)

PnL1 = PnL(positions["Position in treasury"].loc[204046],treasury_comparison['dirty price'].loc[204046],treasury_comparison["modified duration"].loc[204046],-0.5)
PnL2 = PnL(positions["Position in treasury"].loc[204047],treasury_comparison['dirty price'].loc[204047],treasury_comparison["modified duration"].loc[204047],0.5)

returns['PnL in $ for 0.5bp change in both treasuries ytm'] = [PnL1,PnL2,PnL1+PnL2]

returns

,PnL in $ for 0.5bp change in both treasuries ytm
kytreasno,
204046,12843.6235
204047,12865.4472
Total PnL,25709.0708


Suppose the spread in YTM between the two securities disappears, due to a symmetric move of roughly ~17bps in each security's YTM. What is the PnL? (This is just a linearly scaling of your prior answer for a 1bp convergence.) 

**Answer:** 0.17% change in ytm for each security is what we will use to get the PnL for this case.

In [130]:
returns = pd.DataFrame()
returns['kytreasno'] = [204046,204047,'Total PnL']
returns.set_index('kytreasno',inplace=True)

PnL1 = PnL(positions["Position in treasury"].loc[204046],treasury_comparison['dirty price'].loc[204046],treasury_comparison["modified duration"].loc[204046],-17)
PnL2 = PnL(positions["Position in treasury"].loc[204047],treasury_comparison['dirty price'].loc[204047],treasury_comparison["modified duration"].loc[204047],17)

returns['PnL in $ for 17bp change in both treasuries ytm'] = [PnL1,PnL2,PnL1+PnL2]

returns

,PnL in $ for 17bp change in both treasuries ytm
kytreasno,
204046,436683.1996
204047,437425.2063
Total PnL,874108.4059


## 1.4 Result in 2008

Calculate the profit (or loss) on the position on the following two dates:
* 2008-11-25
* 2008-12-16

To calculate the pnl on each date, simply use the prices of the securities on those dates along with your position sizes, ($n_i, n_j$). No coupon is being paid in November or December, so all you need is the "dirty" price on these two dates.

Does the pnl make sense (approximately) given your results in 1.3 with regard to the sensitivity of pnl to moves in the YTM spread?



In [127]:
dates = [quote_date,pd.to_datetime("2008-11-25"),pd.to_datetime("2008-12-16")]

treasury_quotes = treasury_specific.loc[dates][['kytreasno','tdbid','tdask','tdnomprc','tdaccint','tdretnua','tdyld','tdduratn']]
treasury_quotes['dirty price'] = (treasury_quotes['tdbid']+treasury_quotes['tdask'])/2 + treasury_quotes['tdaccint']
treasury_quotes['tdyld'] = treasury_quotes['tdyld']*365.25

init_price_204046 = treasury_comparison['dirty price'].loc[204046]
init_price_204047 = treasury_comparison['dirty price'].loc[204047]

treasury_quotes.reset_index(inplace=True)
treasury_quotes.set_index('kytreasno',inplace=True)
treasury_quotes.sort_index(inplace=True)

PnL1 = n1*(treasury_quotes['dirty price'].loc[204046]-init_price_204046)
PnL2 = n2*(treasury_quotes['dirty price'].loc[204047]-init_price_204047)
PnLs = pd.concat([PnL1,PnL2],axis=0)
treasury_quotes['PnL in $'] = PnLs

merged_df = pd.merge(treasury_quotes, treasury_quotes, on='caldt')
merged_df.drop([0,2,3,4,6,7,8,10,11],inplace=True)
ytm_spread = pd.DataFrame()
ytm_spread['ytm spread'] = (merged_df['tdyld_x']-merged_df['tdyld_y'])*10000

PnL_combined = pd.DataFrame(treasury_quotes.groupby('caldt').sum()['PnL in $'])

PnL_combined['ytm spread (bp)'] = list(ytm_spread['ytm spread'])
PnL_combined

,PnL in $,ytm spread (bp)
caldt,,
2008-11-04,0.0000,33.9031
2008-11-25,-499072.8163,52.9938
2008-12-16,-1049332.7543,71.6619


## Let us try to look at what we got from 1.3:

In this case, the ytm spread has increased in bp instead of decreasing and so it makes sense that the total PnL is negative since we had positive PnL for ytm spread decreasing. Let us assume the spread is caused by equal movements in both to see total PnL with that formula. The values do come pretty close even from the approximation!

In [134]:
returns1 = pd.DataFrame()
returns1['PnL approximation'] = ['Total PnL (2008-11-25)','Total PnL (2008-12-16)']
returns1.set_index('PnL approximation',inplace=True)

PnL1 = PnL(positions["Position in treasury"].loc[204046],treasury_comparison['dirty price'].loc[204046],treasury_comparison["modified duration"].loc[204046],9.5)
PnL2 = PnL(positions["Position in treasury"].loc[204047],treasury_comparison['dirty price'].loc[204047],treasury_comparison["modified duration"].loc[204047],-9.5)

PnL3 = PnL(positions["Position in treasury"].loc[204046],treasury_comparison['dirty price'].loc[204046],treasury_comparison["modified duration"].loc[204046],19)
PnL4 = PnL(positions["Position in treasury"].loc[204047],treasury_comparison['dirty price'].loc[204047],treasury_comparison["modified duration"].loc[204047],-19)

returns1['PnL in $ for approx bp change in both treasuries ytm'] = [PnL1+PnL2,PnL3+PnL4]

returns1

,PnL in $ for approx bp change in both treasuries ytm
PnL approximation,
Total PnL (2008-11-25),-488472.3445
Total PnL (2008-12-16),-976944.6889


## 1.5 Examining the Trade through June 2009

Calculate the pnl of the trade for the following dates:
* 2009-01-27
* 2009-03-24
* 2009-06-16

Did the trade do well or poorly in the first six months of 2009?

In [142]:
dates = [quote_date,pd.to_datetime("2009-01-27"),pd.to_datetime("2009-03-24"),pd.to_datetime("2009-06-16")]


treasury_quotes = treasury_specific.loc[dates][['kytreasno','tdbid','tdask','tdnomprc','tdaccint','tdretnua','tdyld','tdduratn']]
treasury_quotes['dirty price'] = (treasury_quotes['tdbid']+treasury_quotes['tdask'])/2 + treasury_quotes['tdaccint']
treasury_quotes['tdyld'] = treasury_quotes['tdyld']*365.25

init_price_204046 = treasury_comparison['dirty price'].loc[204046]
init_price_204047 = treasury_comparison['dirty price'].loc[204047]

treasury_quotes.reset_index(inplace=True)
treasury_quotes.set_index('kytreasno',inplace=True)
treasury_quotes.sort_index(inplace=True)

coupon_date1 = maturity_date.replace(year=2009)
coupon_date2 = coupon_date1 -timedelta(days=180)
coupon_payment = n1*treasury_comparison['coupon rate'].loc[204046] + n2*treasury_comparison['coupon rate'].loc[204047]

PnL1 = n1*(treasury_quotes['dirty price'].loc[204046]-init_price_204046)
PnL2 = n2*(treasury_quotes['dirty price'].loc[204047]-init_price_204047)
PnLs = pd.concat([PnL1,PnL2],axis=0)
treasury_quotes['PnL in $'] = PnLs

PnL_combined = pd.DataFrame(treasury_quotes.groupby('caldt').sum()['PnL in $'])

conditions = [PnL_combined.index >= coupon_date1, PnL_combined.index >= coupon_date2]

values = [coupon_payment,coupon_payment/2]

PnL_combined['Coupon payment in $'] = np.select(conditions, values, default=0)
PnL_combined['Total PnL in $'] = PnL_combined['PnL in $'] + PnL_combined['Coupon payment in $']

merged_df = pd.merge(treasury_quotes, treasury_quotes, on='caldt')
merged_df.drop([0,2,3,4,6,7,8,10,11,12,14,15],inplace=True)
ytm_spread = pd.DataFrame()
ytm_spread['ytm spread'] = (merged_df['tdyld_x']-merged_df['tdyld_y'])*10000

PnL_combined['ytm spread (bp)'] = list(ytm_spread['ytm spread'])
PnL_combined


,PnL in $,Coupon payment in $,Total PnL in $,ytm spread (bp)
caldt,,,,
2008-11-04,0.0000,0.0000,0.0000,33.9031
2009-01-27,20657.3597,0.0000,20657.3597,35.3588
2009-03-24,-545508.2272,966145.8257,420637.5985,22.0305
2009-06-16,234164.9146,966145.8257,1200310.7402,-7.1405


Calculate the YTM spreads on these dates. Does the YTM spread correspond to pnl roughly as we would expect based on the calculation in 1.3?

In [143]:
returns1 = pd.DataFrame()
returns1['PnL approximation'] = ['Total PnL (2009-01-27)','Total PnL (2009-03-24)','Total PnL (2009-06-16)']
returns1.set_index('PnL approximation',inplace=True)

PnL1 = PnL(positions["Position in treasury"].loc[204046],treasury_comparison['dirty price'].loc[204046],treasury_comparison["modified duration"].loc[204046],0.8)
PnL2 = PnL(positions["Position in treasury"].loc[204047],treasury_comparison['dirty price'].loc[204047],treasury_comparison["modified duration"].loc[204047],-0.8)

PnL3 = PnL(positions["Position in treasury"].loc[204046],treasury_comparison['dirty price'].loc[204046],treasury_comparison["modified duration"].loc[204046],-6)
PnL4 = PnL(positions["Position in treasury"].loc[204047],treasury_comparison['dirty price'].loc[204047],treasury_comparison["modified duration"].loc[204047],6)

PnL5 = PnL(positions["Position in treasury"].loc[204046],treasury_comparison['dirty price'].loc[204046],treasury_comparison["modified duration"].loc[204046],-20.5)
PnL6 = PnL(positions["Position in treasury"].loc[204047],treasury_comparison['dirty price'].loc[204047],treasury_comparison["modified duration"].loc[204047],20.5)


returns1['PnL in $ for approx bp change in both treasuries ytm'] = [PnL1+PnL2,PnL3+PnL4,PnL5+PnL6]

returns1

,PnL in $ for approx bp change in both treasuries ytm
PnL approximation,
Total PnL (2009-01-27),-41134.5132
Total PnL (2009-03-24),308508.8491
Total PnL (2009-06-16),1054071.9012


***

# 2 Hedging Duration

Import `treasury_ts_issue_duration_(207392, 207391, 207404).xlsx`.

I suggest using code such as

* `tsdata = pd.read_excel(filepath_tsdata,sheet_name='ts')`

* `tsdata.columns = tsdata.columns.str.upper()`

* `px = tsdata.pivot_table(index='CALDT',columns='KYTREASNO',values='TDASK').dropna()`
* `duration = tsdata.pivot_table(index='CALDT',columns='KYTREASNO',values='TDDURATN').dropna()`



In [149]:
tsdata = pd.read_excel("treasury_ts_issue_duration_(207392, 207391, 207404).xlsx",sheet_name='ts')
tsdata.columns = tsdata.columns.str.upper()
px = tsdata.pivot_table(index='CALDT',columns='KYTREASNO',values='TDASK').dropna()
duration = tsdata.pivot_table(index='CALDT',columns='KYTREASNO',values='TDDURATN').dropna()

px.tail()

KYTREASNO,207391,207392,207404
CALDT,,,
2023-11-24,85.8594,63.1875,96.9766
2023-11-27,86.2031,63.9375,97.0078
2023-11-28,86.6328,64.3438,97.0391
2023-11-29,87.0234,65.1797,97.0781
2023-11-30,86.7344,64.5000,97.0859


In [150]:
duration.tail()

KYTREASNO,207391,207392,207404
CALDT,,,
2023-11-24,1986.4610,6425.5089,279.8339
2023-11-27,1983.7328,6443.2128,276.8339
2023-11-28,1983.0624,6453.3066,275.8340
2023-11-29,1982.3803,6474.8527,274.8343
2023-11-30,1981.1435,6455.4548,273.8343


## 2.1 

Suppose you have a portfolio of 10,000 USD long in security `207391` on the last day of the sample.

If you want to manage interest rate exposure using duration, how large of a short position should you hold in `207392`?

(Duration is the column `TDDURATN` in the raw data.)


In [173]:
n1 = 10000/px[207391]     #Number of units held in security 207391
n2 = -n1*duration[207391]/duration[207392]

rate_hedge = pd.concat([n1,n2],axis=1)
rate_hedge.columns = ['Units of 207391 held','Units of 207392 held']
rate_hedge['Dollar value of 207391 held'] = rate_hedge['Units of 207391 held']*px[207391]
rate_hedge['Dollar value of 207392 held'] = rate_hedge['Units of 207392 held']*px[207392]
rate_hedge['Holding value in $'] = rate_hedge['Dollar value of 207391 held'] + rate_hedge['Dollar value of 207392 held']

rate_hedge


,Units of 207391 held,Units of 207392 held,Dollar value of 207391 held,Dollar value of 207392 held,Holding value in $
CALDT,,,,,
2019-08-29,98.8112,-41.0730,10000.0000,-4372.9884,5627.0116
2019-08-30,98.8264,-41.0694,10000.0000,-4373.8917,5626.1083
2019-09-03,98.4388,-40.8701,10000.0000,-4361.9298,5638.0702
2019-09-04,98.5222,-40.9265,10000.0000,-4351.9565,5648.0435
2019-09-05,99.3712,-41.4024,10000.0000,-4326.5456,5673.4544
...,...,...,...,...,...
2023-11-24,116.4695,-36.0068,10000.0000,-2275.1810,7724.8190
2023-11-27,116.0051,-35.7156,10000.0000,-2283.5649,7716.4351
2023-11-28,115.4297,-35.4709,10000.0000,-2282.3279,7717.6721



## 2.2

Step through the time-series, doing the following:

* Starting at the end of the first day, set the hedged position according to the relative given durations.
* Use the second day's price data to evaluate the net profit or loss of the hedged position.
* Reset the the hedged position using the end-of-second-day durations. Again fix the long position of security `207391` to be 10,000.
* Repeat throughout the timeseries.

Report 
* the total profit (or loss.)
* the mean, standard deviation, min, and max of the daily profit or loss.

In [175]:
price_change = (px - px.shift(1))
price_change = price_change.iloc[1:]

rate_hedge = rate_hedge.shift(1)
rate_hedge = rate_hedge.iloc[1:]

PnLs = pd.DataFrame()

PnLs[['Units of 207391 held','Units of 207392 held']] = rate_hedge[['Units of 207391 held','Units of 207392 held']]
PnLs[['price change 207391','price change 207392']] = price_change[[207391,207392]]

PnLs['PnL 207391'] = PnLs['Units of 207391 held']*PnLs['price change 207391']
PnLs['PnL 207392'] = PnLs['Units of 207392 held']*PnLs['price change 207392']

PnLs['Daily PnL'] = PnLs['PnL 207391']+PnLs['PnL 207392']
PnLs['Total Cumulative PnL'] = PnLs['Daily PnL'].cumsum()
PnLs

,Units of 207391 held,Units of 207392 held,price change 207391,price change 207392,PnL 207391,PnL 207392,Daily PnL,Total Cumulative PnL
CALDT,,,,,,,,
2019-08-30,98.8112,-41.0730,-0.0156,0.0312,-1.5439,-1.2835,-2.8275,-2.8275
2019-09-03,98.8264,-41.0694,0.3984,0.2266,39.3762,-9.3048,30.0714,27.2439
2019-09-04,98.4388,-40.8701,-0.0859,-0.3906,-8.4596,15.9649,7.5053,34.7492
2019-09-05,98.5222,-40.9265,-0.8672,-1.8359,-85.4372,75.1385,-10.2987,24.4505
2019-09-06,99.3712,-41.4024,0.0000,0.5312,0.0000,-21.9950,-21.9950,2.4555
...,...,...,...,...,...,...,...,...
2023-11-24,116.1735,-35.8412,-0.2188,-0.6562,-25.4130,23.5208,-1.8922,-117.8791
2023-11-27,116.4695,-36.0068,0.3438,0.7500,40.0364,-27.0051,13.0313,-104.8478
2023-11-28,116.0051,-35.7156,0.4297,0.4062,49.8459,-14.5095,35.3365,-69.5114


In [174]:
daily_pnl_stats = pd.DataFrame()
daily_pnl_stats['Mean'] = [PnLs['Daily PnL'].mean()]
daily_pnl_stats['Std dev'] = [PnLs['Daily PnL'].std()]
daily_pnl_stats['Min'] = [PnLs['Daily PnL'].min()]
daily_pnl_stats['Max'] = [PnLs['Daily PnL'].max()]

daily_pnl_stats["Daily PnL stats"] = ['PnL analysis']
daily_pnl_stats.set_index("Daily PnL stats",inplace=True)
daily_pnl_stats

,Mean,Std dev,Min,Max
Daily PnL stats,,,,
PnL analysis,-0.0595,23.4512,-126.0785,114.0918


***

# 3 Calculating Duration Via Individual Cashflows

## *Optional*

***No need to submit this problem, but if we discuss it, then you are expected to know it.***

Use the data file `../data/treasury_quotes_2023-07-26.xlsx`.

This data reports duration as `TDDURATN`. It quotes the duration in days, so I recommend dividing by 365 to get the duration in its usual format.

## 3.1 
Set up the cashflow matrix. 


## 3.2
Build a discount curve assuming that the spot rate is 2\% per year, continuously compounded.

Plot the discount curve and the associated spot curve out to 30 years maturity.

*Note, you do not need to properly extract a spot curve and associated discount factors; rather, you are simply assuming a flat term structure of spot rates at 2\% and using that to figure out discount rates.*

## 3.3
For each treasury issue, calculate the duration as the weighted average of the (discounted!) cashflow maturity.

Report the summary statistics of the durations. (Use `.describe()` from pandas.)


## 3.4
How close are your duration estimates to the imputed durations given in the data source, (column `TDDURATN`)?

Report the summary statistics of the imputed durations minus your calculated durations from above.

Why might they be different?

## 3.5
Continue using your assumed discount rates of 2\% to calculate the convexity of each issue.

Report the summary statistics of these convexity calculations.

## 3.6

Re-do the duration and convexity calculations using an extracted discount curve instead of a discount curve based on a constant (arbitrary) spot rate. 

***

# 4 Other Interest-Rate Risks

## *Optional*

***No need to submit this problem, but if we discuss it, then you are expected to know it.***

## 4.1 Other Yield Curve Movements

Use the yield curve time-series data in `'../data/yields.xlsx` to calculate the time-series of the **level**, **slope**, and curvature** factors.

Calculate the yield-curve factors. For each point in time, calculate the following three factors:

$\begin{align}
x^{\text{level}}_t =& \frac{1}{N_{\text{yields}}}\sum_{i=1}^{N_{\text{yields}}} y^{(i)}_t\\
x^{\text{slope}}_t =& y^{(30)}_t - y^{(1)}_t\\
x^{\text{curvature}}_t =& -y^{(1)}_t + 2 y^{(10)}_t - y^{(30)}_t
\end{align}$

Report the mean and volatility of each factor.

Report the correlation matrix of the factors.

## 4.2 Factor Duration

Calculate the factor duration of the treasuries from `treasury_ts_issue_duration_(207392, 207391, 207457).xlsx`.

Run a multivariate regression of the bond prices on all three factors constructed above from the yield factors: level, slope, and curvature. 

Estimate the regression in the form of day-over-day differences for both bond prices and factors. That is, we are using regression to approximate the factor duration equation,

$\begin{align}
\frac{dP}{P} = \beta_1 dz_1 + \beta_2 dz_2 + \beta_3 dz_3 + \epsilon
\end{align}$

Report the betas for each of these factors, for each of the bond prices.

***